In [1]:
import pandas
import numpy as np

In [2]:
li=[]
for i in range(10,20):
    print('{}{}'.format(i,i+1))
    boxscore=pandas.read_csv('./boxscore/gamehunx{}{}.csv'.format(i,i+1),header=None)
    li.append(boxscore)
boxscore = pandas.concat(li, axis=0, ignore_index=True)

1011
1112
1213
1314
1415
1516
1617
1718
1819
1920


In [3]:
boxscore.columns=["CODE","team","GM","Min","PF","DF","2Pm","2Pa","2PP","3Pm","3Pa","3PP","FTm","Fta","FTP","ORB","DRB","TRB","BS","SR","TOV","ST","ASS","PTS", "VALU","OP","OER","ASTO","VIR","extra1","date","extra3","nat"]

In [4]:
boxscore=boxscore.drop(axis=1,labels=['VALU','OP','OER','ASTO','VIR','extra3','GM','CODE','nat'])

In [5]:
f = dict.fromkeys(boxscore.columns, 'sum')
f.update(
    dict.fromkeys(['2PP','3PP','FTP','date'], 'mean'))

In [6]:
result=boxscore.groupby(['extra1','date','team']).agg(f)

In [7]:
result=result.drop(axis=1,labels=['extra1','team','Min','date'])
result.columns

Index(['PF', 'DF', '2Pm', '2Pa', '2PP', '3Pm', '3Pa', '3PP', 'FTm', 'Fta',
       'FTP', 'ORB', 'DRB', 'TRB', 'BS', 'SR', 'TOV', 'ST', 'ASS', 'PTS'],
      dtype='object')

In [8]:
result=result.reset_index()

In [9]:
result=result.sort_values(["date",'extra1'])

In [10]:
result

,extra1,date,team,PF,DF,2Pm,2Pa,2PP,3Pm,3Pa,...,FTP,ORB,DRB,TRB,BS,SR,TOV,ST,ASS,PTS
1240,900,20101009,9005,18,17,23,32,607.666667,12,31,...,541.666667,6,16,22,5,1,9,20,27,97
1241,900,20101009,40051,17,18,22,42,297.090909,3,10,...,382.272727,12,12,24,1,5,14,11,4,67
1244,901,20101009,9007,17,27,23,35,421.545455,6,24,...,431.818182,10,26,36,4,3,11,8,17,86
1245,901,20101009,9013,27,17,26,49,244.333333,5,17,...,229.166667,7,19,26,3,4,11,9,13,77
1248,902,20101009,9003,14,18,26,40,577.750000,14,26,...,494.416667,11,25,36,0,0,12,9,21,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,1302,20200307,9230,15,16,24,37,316.916667,7,23,...,429.166667,3,19,22,0,0,9,9,20,82
2618,1303,20200307,9002,22,20,20,35,467.083333,8,29,...,347.250000,10,23,33,4,3,15,7,16,74
2619,1303,20200307,9006,20,22,24,41,378.800000,6,23,...,362.500000,8,26,34,3,4,10,6,20,79
2622,1304,20200307,6015,21,20,20,39,279.166667,3,23,...,451.416667,8,25,33,2,0,14,8,16,66


In [11]:
data=pandas.DataFrame(columns=['gameid',"teamid","2PP","3PP","DRB%","ASS%","TOV%","Win%","3PTS/PTS","FTP"])

In [12]:
data['gameid']=result['extra1']
data["teamid"]=result["team"]
data['date']=result['date']
data["2PP"]=result["2PP"]/10
data["3PP"]=result["3PP"]/10
data["FTP"]=result["FTP"]/10
data["3PTS/PTS"]=result["3Pm"]*3/result["PTS"]*100
data["FT/PTS"]=result["FTm"]/result["PTS"]*100
data["Poss"]=0.96*(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']-result['ORB'])
data["ASS%"]=result['ASS']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
data["TOV%"]=result['TOV']/(result['2Pa']+result['3Pa']+0.44*result['Fta']+result['TOV']+result['ASS'])*100
data['OffRating']=100*result['PTS']/data['Poss']
data['DF']=result['DF']

In [13]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [14]:
lista1=[]
lista2=[]
lista3=[]
lista4=[]
lista5=[]
lista6=[]
lista7=[]
lista8=[]
for (i1, row1), (i2, row2) in pairwise(result.sort_values(['date','extra1']).iterrows()):    
    if(row1['extra1']==row2['extra1']):
        if(row1['PTS']>row2['PTS']):
            lista1.append(100)
            lista1.append(0)
        else:
            lista1.append(0)
            lista1.append(100)
        lista2.append(row1['DRB']/(row2['Fta']-row2['FTm']+row2['2Pa']-row2['2Pm']+row2['3Pa']-row2['3Pm'])*100)
        lista2.append(row2['DRB']/(row1['Fta']-row1['FTm']+row1['2Pa']-row1['2Pm']+row1['3Pa']-row1['3Pm'])*100)
        lista3.append(row2['PTS']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista3.append(row1['PTS']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row1['ST']/(0.96*(row2['2Pa']+row2['3Pa']+0.44*row2['Fta']+row2['TOV']-row2['ORB']))*100)
        lista4.append(row2['ST']/(0.96*(row1['2Pa']+row1['3Pa']+0.44*row1['Fta']+row1['TOV']-row1['ORB']))*100)
        lista5.append(row1['ORB']/(row1['ORB']+row2['DRB'])*100)
        lista5.append(row2['ORB']/(row2['ORB']+row1['DRB'])*100)
        lista6.append(row1['BS']/(row2['2Pa']+row2['3Pa'])*100)
        lista6.append(row2['BS']/(row1['2Pa']+row1['3Pa'])*100)
        lista7.append(row2['3PP']/10)
        lista7.append(row1['3PP']/10)
        lista8.append(row2['2PP']/10)
        lista8.append(row1['2PP']/10)

In [15]:
data['Win%']=lista1
data['DRB%']=lista2
data['DefRating']=lista3
data['ST%']=lista4
data['ORB%']=lista5
data['BL%']=lista6
data['Opp_3PP']=lista7
data['Opp_2PP']=lista8

In [16]:
data=data.sort_values(['teamid','date','gameid'])
data=data.reset_index()
data=data.drop(axis=1,columns=['index'])

In [17]:
szezon=[]
date=20100830
for j,r in data.iterrows():
    for i in range(0,10):
        if((r['date']>date+i*10000) &(r['date']<date+(i+1)*10000)):
            szezon.append(i)
            break

In [18]:
data['szezon']=szezon

In [21]:
#byszezon=data[~data['teamid'].isin([9221,9233,40051,9218])]
byszezon=data
byszezon=byszezon[byszezon['szezon'].isin([0,1,2,3,4])]

In [22]:
elozoevekatlaga=byszezon.groupby(['teamid']).mean()
elozoevekatlaga=elozoevekatlaga.reset_index()
elozoevekatlaga

,teamid,gameid,2PP,3PP,DRB%,ASS%,TOV%,Win%,3PTS/PTS,FTP,...,Poss,OffRating,DF,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP,szezon
0,6015,1140.288462,42.595697,21.959240,59.914784,18.156450,10.872723,82.692308,29.474565,34.226799,...,70.382031,124.746865,19.057692,105.911842,15.901183,32.079910,5.620480,16.259889,33.616218,0.500000
1,9001,1120.800000,39.356783,17.987237,64.013993,14.807454,12.710589,71.666667,24.741785,32.012452,...,72.988160,117.650282,19.925000,101.492982,15.509625,32.229702,5.443426,16.076238,33.322914,1.933333
2,9002,1120.475000,40.668429,17.270149,60.407270,15.407434,13.710635,59.166667,23.490759,33.054324,...,72.288320,118.639918,20.708333,113.374905,10.740325,30.200652,3.736210,16.364319,36.046290,1.933333
3,9003,1120.391667,37.847437,19.433118,58.816998,13.922359,11.365979,58.333333,31.058435,28.226679,...,71.236480,118.679581,20.041667,112.731542,12.692971,28.280618,4.012917,15.834923,34.843273,1.933333
4,9004,1119.816667,33.842223,14.381408,61.114329,14.877015,12.804301,53.333333,22.300031,32.562601,...,72.574080,113.675428,20.641667,115.364718,12.482154,27.490029,3.809771,18.331621,38.086919,1.933333
5,9005,1120.333333,34.662875,15.851055,59.097737,13.819774,13.273841,29.166667,27.035320,30.222687,...,69.535040,107.813804,19.666667,120.233740,14.108155,26.818743,2.658429,17.683344,38.988755,1.933333
6,9006,1120.966667,38.265043,18.413283,61.547069,14.747211,12.727951,54.166667,30.450876,31.474288,...,69.016320,115.592828,19.866667,112.446512,11.601659,28.550456,3.436831,17.005899,34.415398,1.933333
7,9007,1120.450000,40.917389,19.548391,62.393214,15.552992,12.194581,75.833333,25.673593,35.882245,...,71.409920,118.629943,19.716667,103.956376,13.004768,32.007042,3.750639,15.889502,33.061896,1.933333
8,9011,1120.558333,33.327924,14.952937,59.471119,12.622033,12.700025,32.500000,24.756136,28.875064,...,71.640000,108.775117,20.525000,118.505682,15.389216,24.862605,3.129533,19.288118,37.963844,1.933333
9,9013,1121.166667,33.078025,14.625654,59.348402,14.631222,13.929214,37.500000,26.227160,33.065955,...,74.097600,110.368489,20.500000,115.517411,11.370803,25.040958,3.112542,16.696311,38.835239,1.933333


In [23]:
data.groupby(['teamid','szezon']).count().groupby('teamid').count()

,gameid,2PP,3PP,DRB%,ASS%,TOV%,Win%,3PTS/PTS,FTP,date,FT/PTS,Poss,OffRating,DF,DefRating,ST%,ORB%,BL%,Opp_3PP,Opp_2PP
teamid,,,,,,,,,,,,,,,,,,,,
6015,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
9001,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9002,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9003,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9004,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9005,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9006,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9007,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
9011,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [302]:
cnt=1
actual_team=0
index=0
lista=[]
for i,r in data.iterrows():
    adatok=[]
    if(r["teamid"]==actual_team):
        if(r['date']>20150830):
            cnt+=1
            if((actual_team in [40010,9217,9233,40000])&(cnt>5)):
                last_five=data[i-5:i]
                season=data[i-min(cnt,20):i]
                eves=data[i-cnt:i]
                adatok.append(r['gameid'])
                adatok.append(actual_team)
                for c in data.columns:
                    if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                        adatok.append(last_five[c].mean())
                    if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                        adatok.append(season[c].mean())
                    if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                        adatok.append(eves[c].mean())
                adatok.append(r['date'])
                lista.append(adatok)
            if(actual_team not in [40010,9217,9233,40000]):
                last_five=data[i-5:i]
                season=data[i-20:i]
                eves=elozoevekatlaga[elozoevekatlaga['teamid']==actual_team]
                adatok.append(r['gameid'])
                adatok.append(actual_team)
                for c in data.columns:
                    if(c  in ['OffRating', 'ASS%','2PP', '3PP','TOV%','DRB%','ST%','BL%']):
                        #print(c,':',last_five[c].mean())
                        adatok.append(last_five[c].mean())
                    if(c  in ['DefRating','Win%','3PTS/PTS', 'FTP','FT/PTS','Poss','Opp_3PP','DF']):
                        #print(c,':',season[c].mean())
                        adatok.append(season[c].mean())
                    if(c  in ['OffRating','Win%','3PTS/PTS', 'ST%','FT/PTS','ORB%','Opp_2PP','TOV%']):
                        #print(c,':',eves[c].mean())
                        adatok.append(eves[c].mean())
                adatok.append(r['date'])
                lista.append(adatok)
    else:
        actual_team=r["teamid"]
        cnt=1
        first=0

In [303]:
len(lista)

1718

In [304]:
data2=pandas.DataFrame(data=lista,columns=['gameid',"teamid",'2PP_g', '3PP_g', 'DRB%_g', 'ASS%_g', 'TOV%_g','TOV%_y', 'Win%_s','Win%_y','3PTS/PTS_s','3PTS/PTS_y', 'FTP_s','FT/PTS_s','FT/PTS_y', 'Poss_s', 'OffRating_g','OffRating_y', 'DF_s','DefRating_s','ST%_g','ST%_y','ORB%_y','BL%_g','Opp_3PP_s','Opp_2PP_y','date'])

In [305]:
tabla=[]
for (i1, row1), (i2, row2) in pairwise(data2.sort_values(['date','gameid']).iterrows()):
    sor=[]
    if(row1['gameid']==row2['gameid']):
        for c in data2.columns:
            sor.append(row1[c])
        for c in data2.columns:
            if(c not in ['gameid','date']):
                sor.append(row2[c])
        tabla.append(sor)

In [265]:
len(tabla)

729

In [358]:
test=pandas.DataFrame(data=tabla,columns=['gameid',"teamid_1",'2PP_g_1', '3PP_g_1', 'DRB%_g_1', 'ASS%_g_1', 'TOV%_g_1','TOV%_y_1', 'Win%_s_1','Win%_y_1','3PTS/PTS_s_1','3PTS/PTS_y_1', 'FTP_s_1','FT/PTS_s_1','FT/PTS_y_1', 'Poss_s_1', 'OffRating_g_1','OffRating_y_1', 'DF_s_1','DefRating_s_1','ST%_g_1','ST%_y_1','ORB%_y_1','BL%_g_1','Opp_3PP_s_1','Opp_2PP_y_1','date',"teamid_2",'2PP_g_2', '3PP_g_2', 'DRB%_g_2', 'ASS%_g_2', 'TOV%_g_2','TOV%_y_2', 'Win%_s_2','Win%_y_2','3PTS/PTS_s_2','3PTS/PTS_y_2', 'FTP_s_2','FT/PTS_s_2','FT/PTS_y_2', 'Poss_s_2', 'OffRating_g_2','OffRating_y_2', 'DF_s_2','DefRating_s_2','ST%_g_2','ST%_y_2','ORB%_y_2','BL%_g_2','Opp_3PP_s_2','Opp_2PP_y_2'])

In [50]:
test.to_csv('test5.csv',index=False)

In [163]:
#legközelebb elég csak beolvasni

In [359]:
results=pandas.read_csv('allgames.csv')

In [342]:
test=pandas.read_csv('test.csv')

In [360]:
merge=pandas.merge(results,test,on=['date',"gameid"],how='inner')

In [361]:
merge=merge.drop(axis=1,columns=['team1','team2'])

In [362]:
merge

,gameid,date,won,teamid_1,2PP_g_1,3PP_g_1,DRB%_g_1,ASS%_g_1,TOV%_g_1,TOV%_y_1,...,OffRating_g_2,OffRating_y_2,DF_s_2,DefRating_s_2,ST%_g_2,ST%_y_2,ORB%_y_2,BL%_g_2,Opp_3PP_s_2,Opp_2PP_y_2
0,1.0,20151002.0,9002.0,9002.0,33.308333,20.745000,57.382955,12.505598,13.431058,13.710635,...,109.256291,112.566533,20.85,99.198944,22.221129,15.519533,29.169716,4.198585,15.611591,33.644362
1,2.0,20151002.0,9003.0,9003.0,35.587727,25.207576,63.195352,15.796283,8.554508,11.365979,...,116.655336,110.368489,21.75,118.125831,7.399379,11.370803,25.040958,2.341774,16.447583,38.835239
2,3.0,20151003.0,9007.0,9004.0,40.517727,11.299394,64.195754,18.430628,10.994301,12.804301,...,112.494840,118.629943,20.55,97.297803,10.802425,13.004768,32.007042,2.464621,17.834143,33.061896
3,4.0,20151003.0,9011.0,9011.0,38.310000,13.156818,63.950926,12.680857,15.176753,12.700025,...,108.375657,102.141270,22.75,118.839116,9.320800,8.523174,27.558333,1.751748,15.892879,36.522273
4,5.0,20151003.0,9006.0,9006.0,40.456667,16.730000,59.965955,13.914138,14.439003,12.727951,...,110.058565,112.299822,20.95,110.070585,12.084063,12.944563,26.604648,2.407755,16.315030,36.201250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,1150.0,20190406.0,9213.0,9005.0,33.906667,24.131667,60.342422,14.038413,11.944251,13.273841,...,110.126488,112.299822,20.00,110.058647,9.294047,12.944563,26.604648,3.500005,16.640417,36.201250
535,1151.0,20190406.0,9003.0,9003.0,39.928333,24.350000,55.253400,16.170592,9.003446,11.365979,...,112.978479,118.629943,19.55,109.768089,6.541256,13.004768,32.007042,5.251906,18.596174,33.061896
536,1152.0,20190406.0,9800.0,9004.0,38.610152,15.927576,61.304213,15.918888,12.366808,12.804301,...,110.301848,112.566533,19.90,114.180400,7.484431,15.519533,29.169716,2.582279,17.740265,33.644362
537,1153.0,20190406.0,9006.0,9006.0,32.967242,21.438242,57.097886,16.555643,10.153468,12.727951,...,112.311678,110.368489,20.40,113.221581,8.541584,11.370803,25.040958,2.808089,16.730152,38.835239


In [363]:
w=[]
for i,r in merge.iterrows():
    if(r['won']==r['teamid_1']):
        w.append(1)
    else:
        w.append(2)
merge['won']=w

In [364]:
merge=merge.drop(axis=1,columns=['teamid_1','teamid_2','gameid'])

In [365]:
labels=np.array(merge['won'])

In [366]:
merge=merge.drop(axis=1,columns=['date','won'])

In [367]:
merge = np.array(merge)

In [40]:
from sklearn.model_selection import train_test_split

In [368]:
train_data, test_data, train_labels, test_labels = train_test_split(merge, labels, test_size = 0.1, random_state = 42)

In [369]:
print('Training Features Shape:', train_data.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_data.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (485, 48)
Training Labels Shape: (485,)
Testing Features Shape: (54, 48)
Testing Labels Shape: (54,)


In [60]:
from sklearn.ensemble import RandomForestClassifier

In [355]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [370]:
rf.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [63]:
rf.score(test_data,test_labels)

0.7037037037037037